# Task 1 - Quantum Computing

Note, at time of writing task 1 (06/03/2023), there is a bug in the pennylane library which produces an AttributeError when calling the qml.RX,qml.RY functions, detailed at: 

https://github.com/PennyLaneAI/pennylane/issues/3867

Hence, in test-env using autoray 0.6.0

In [3]:
import pennylane as qml
from pennylane import numpy as np

In [17]:
task1Device = qml.device('default.qubit', wires=5)

def wireWiseHadamard(wires: list):
    '''Performs hadamard operations on each qubit in the list of wires'''
    for wire in wires:
        qml.Hadamard(wire)
        
def pairWiseCNOT(device):
    '''performs CNOT operations on pairwise wires in the device for ascending labels, i.e for N 
    wires, performs CNOT on (0,1), (1,2), ... , (N-1, N)'''
    
    wires = list(device.wires)
    
    for wire in range(len(wires)-1):
        qml.CNOT(wires=[wire, wire+1])
        

        
@qml.qnode(task1Device)
def circuit1(xAxisRotationAngle: float):
    
    wireWiseHadamard(list(task1Device.wires))
    pairWiseCNOT(task1Device)
    qml.SWAP(wires=[0,4])
    qml.RX(xAxisRotationAngle, wires=2) #arbitrarily chosen 3rd qubit
    return qml.expval(qml.PauliZ(wires=0))

    
drawer = qml.draw(circuit1)
print(drawer(np.pi/2))


    
    
    

0: ──H─╭●──────────╭SWAP───────────┤  <Z>
1: ──H─╰X─╭●───────│───────────────┤     
2: ──H────╰X─╭●────│──────RX(1.57)─┤     
3: ──H───────╰X─╭●─│───────────────┤     
4: ──H──────────╰X─╰SWAP───────────┤     


## Task 1 - Circuit 2 Plan

hadamard gate as in circuit 1
rotation as in 1
hadamard gates as usual, using the wirewise function

swap test - use operations in https://zenodo.org/record/3164691#.YigdGt9MHrB 

assuming one run of the swap test as there is no reference to number of tests and